In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("/kaggle/input/churn-in-telecoms-dataset/bigml_59c28831336c6604c800002a.csv")

In [ ]:
df.describe()

1,2 Задача классификации потому, что целевая переменная chunk - категориальная


In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
sns.catplot(x="churn", kind="count", palette="ch:.25", data=df)

3. Классы не збалансированы


In [ ]:
df["area code"].value_counts()

In [ ]:
df["state"].value_counts()

Штатов слишком много, для такого количества данных и они не вносят явного вклада в таргет (дальше будет видно). Номера телефоно вообще не нужны.


In [ ]:
df_1 = df.drop(["state","phone number"], axis = 1)

In [ ]:
df_2 = pd.concat([df_1, pd.get_dummies(df_1["area code"])], axis = 1)

In [ ]:
df_2

In [ ]:
y = df_2["churn"].map({False:0,True:1})

In [ ]:
df_3 = df_2.drop(["area code", "churn"], axis = 1)

In [ ]:
df_numeric = df_3.copy()

In [ ]:
df_numeric["international plan"] = df_3["international plan"].map({"yes":1,"no":0})

In [ ]:
df_numeric["voice mail plan"] = df_3["voice mail plan"].map({"yes":1,"no":0})

In [ ]:
y

In [ ]:
df_numeric

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(df_numeric)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))

Хоть акураси и большой - видно что есть очень много ошибок 2 рода

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(knn,X,y, cv = kf, scoring = "accuracy"))

accuracy как метрика неправильная ибо классы сильно несбалансированные. Правильней использовать F1 score

In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(knn,X,y, cv = kf, scoring = "f1"))

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
params = {"n_neighbors" : np.arange(1, 51, 2)}
cv = GridSearchCV(knn, params, cv = kf, scoring="f1")
cv.fit(X,y)

In [ ]:
cv.best_estimator_

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

При количестве соседей равном 3 модель показала лучшие реультаты

In [ ]:
cv_results = pd.DataFrame(cv.cv_results_)

In [ ]:
cv_results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(cv_results["param_n_neighbors"],cv_results["mean_test_score"])

k_best = 3

In [ ]:
p_params = {"p": np.linspace(1,10,200)}
knn = KNeighborsClassifier(n_neighbors=3, weights = "distance", n_jobs = -1)
cv = GridSearchCV(knn, p_params, cv = kf, scoring="f1", verbose = 100)
cv.fit(X,y)

In [ ]:
cv_results = pd.DataFrame(cv.cv_results_)


In [ ]:
plt.plot(cv_results["param_p"],cv_results["mean_test_score"])

In [ ]:
cv.best_estimator_

In [ ]:
cv.best_score_

p_best = 1.4974874371859297

Попробоуем другие модели

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier

radius_neighbors_classifier = RadiusNeighborsClassifier(radius=10)
radius_neighbors_classifier.fit(X_train, y_train)

radius_neighbors_classifier.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import NearestCentroid

nearest_centroid = NearestCentroid()
nearest_centroid.fit(X_train, y_train)
nearest_centroid.score(X_test, y_test)

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(kernel="rbf")
scores = cross_val_score(svc,X,y,cv=kf, scoring="f1")

In [ ]:
scores.mean()

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
forest = RandomForestClassifier(n_estimators = 500, max_depth=5, random_state=42)

In [ ]:
scores = cross_val_score(forest, X,y, cv=kf, scoring = "f1")

In [ ]:
scores.mean()

Проведём анализ важности фич и корелляционный анализ.

In [ ]:
corrmat = df_numeric.corr(method='spearman')
f, ax = plt.subplots(figsize=(8, 8))

# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=1., square=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

In [ ]:
import xgboost as xgb

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(df_numeric, y)
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(clf, max_num_features=50, height=0.8, ax=ax)
plt.show()

Можно сделать вывод, что регионы не важны для модели

In [ ]:
xgboostClassifier = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
scores = cross_val_score(xgboostClassifier,X,y,cv=kf, scoring="f1")

In [ ]:
scores.mean()

Хороший показатель, попробуем его улучшить. Попробуем наривать датасет, может визульно будет что-то видно. Так как размерность боольшая - ее нужно уменьшить.

In [ ]:
from sklearn.manifold import TSNE
X_1 = TSNE(n_components=2).fit_transform(df_numeric)
plt.scatter(X_1[:,0],X_1[:,1], c = y)

In [ ]:
from sklearn.manifold import TSNE
X_1 = TSNE(n_components=2).fit_transform(X)
plt.scatter(X_1[:,0],X_1[:,1], c = y)

In [ ]:
from sklearn.decomposition import PCA
X_2 = PCA(n_components=2).fit_transform(df_numeric)
plt.scatter(X_2[:,0],X_2[:,1], c = y)

In [ ]:
from sklearn.decomposition import PCA
X_3 = PCA(n_components=2).fit_transform(X)
plt.scatter(X_3[:,0],X_3[:,1], c = y)

Однако это ничего полезного не дало

Подберём гипер параметры екстремальному градиентному бустингу деревьей для улучшения её качества

In [ ]:
params = {"max_depth" : np.arange(3, 11), "n_estimators":[100,150,200,250,300,400]}
xgboostClassifier = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
cv = GridSearchCV(xgboostClassifier, params, cv = kf, scoring="f1", verbose = 100)
cv.fit(X,y)

In [ ]:
cv_results = pd.DataFrame(cv.cv_results_)

In [ ]:
plt.plot(cv_results["param_max_depth"],cv_results["mean_test_score"])

In [ ]:
plt.plot(cv_results["param_n_estimators"],cv_results["mean_test_score"])

In [ ]:
cv.best_estimator_

In [ ]:
cv.best_score_

Как результат лучшей моделью для данного датасета оказалась XGBoostClassifier с параметрами max_depth = 7, n_estimators = 100

Это с учётом того что были удалены штаты из-за предположения что штатов много, а даннах мало и модель просто не сможет нормально учесть штаты. Пробуем со штатами.

In [ ]:
df_states = df.copy()

In [ ]:
df_states = df_states.drop("phone number", axis = 1)

In [ ]:
df_states

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
df_states['state'] = label_encoder.fit_transform(df_states['state'])
df_states['international plan'] = label_encoder.fit_transform(df_states['international plan'])
df_states['voice mail plan'] = label_encoder.fit_transform(df_states['voice mail plan'])

In [ ]:
df_states = df_states.drop("churn", axis = 1)

In [ ]:
X = scaler.fit_transform(df_states)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gboost= GradientBoostingClassifier(max_depth=7, n_estimators = 100)
score = cross_val_score(gboost,X,y,cv = kf, scoring = "f1")

In [ ]:
scores.mean()

Возможно не зватает просто данных для обучения, увеличю количество fold

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
gboost= GradientBoostingClassifier(max_depth=7, n_estimators = 100)
score = cross_val_score(gboost,X,y,cv = kf, scoring = "f1")
score.mean()

In [ ]:
xgboost = xgb.XGBClassifier(max_depth=7, n_estimators=100, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
score = cross_val_score(gboost,X,y,cv = kf, scoring = "f1")
score.mean()

Добавление штатов как и ожидалось не принесло улучшений

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
from sklearn.metrics import classification_report
xgboost.fit(X_train,y_train)
y_pred = xgboost.predict(X_test)
print('Gradient Boosting Classifier:\n {}\n'.format(classification_report(y_test,y_pred)))

Всё еще есть ошибки 2 рода, но их уже меньше.